# 9

In [87]:
text = "YELLOW SUBMARINE"

In [17]:
class PKCS7:
    def __init__(self, block_size):
        self.block_size = block_size

    def pad(self, text):
        n = self.block_size - len(text) % self.block_size
        return text + bytes([n] * n)
    
    def unpad(self, text):
        n = text[-1]
        return text[:-n]

In [90]:
pkcs7 = PKCS7(20)
pkcs7.pad(text.encode())

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# 10

In [1]:
import base64
key = "YELLOW SUBMARINE"
with open('./cryptopals.com_static_challenge-data_10.txt', 'r') as f:
    file = base64.b64decode(f.read().replace('\n', '').encode())

In [2]:
from utils import xor, PKCS7, AES_CBC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

blockSize = int(algorithms.AES(key.encode()).block_size/8)

In [3]:
aes_cbc = AES_CBC(key = key.encode(),
                  IV = b'\x00'*blockSize)
print(aes_cbc.decrypt(file).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


# 11

In [7]:
import os
from utils import AES_CBC, PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def encryption_oracle(plainText):
    lengthRandomBytes = 5 + int(int.from_bytes(os.urandom(1), byteorder='big')* (6/256))
    plainText = os.urandom(lengthRandomBytes) + plainText + os.urandom(lengthRandomBytes)

    blockSize = 16
    key = os.urandom(blockSize)

    if int.from_bytes(os.urandom(1), byteorder='big')/256 < 0.5:
        cipher = AES_CBC(key, IV=os.urandom(blockSize))
        return cipher.encrypt(plainText)
    else:
        cipher = Cipher(algorithm=algorithms.AES(key),
                        mode=modes.ECB())
        padding = PKCS7(blockSize)
        plainText = padding.pad(plainText)
        encryptor = cipher.encryptor()
        return encryptor.update(plainText)

In [8]:
import base64
key = "YELLOW SUBMARINE"
with open('./cryptopals.com_static_challenge-data_10.txt', 'r') as f:
    file = base64.b64decode(f.read().replace('\n', '').encode())

In [9]:
from utils import check_repetitions_ECB
from collections import Counter

experiments = []
for i in range(0, 1000):
    if check_repetitions_ECB(encryption_oracle(b'0'*256), 16):
        experiments.append('ECB mode')
    else:
        experiments.append('CBC mode')

counts = Counter(experiments)
results = {key: value/len(experiments) for key, value in counts.items()}
print(results)

{'CBC mode': 0.514, 'ECB mode': 0.486}


# 12

In [170]:
import os
import base64
from utils import PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

class Oracle:
    def __init__(self, blockSise):
        self.blockSize = blockSize
        self.key = os.urandom(self.blockSize)
        self.header = base64.b64decode("""
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
""")
        self.cipher = Cipher(
            algorithm=algorithms.AES(self.key),
            mode=modes.ECB()
            )
        self.encryptor = self.cipher.encryptor()
        self.padding = PKCS7(self.blockSize)

    def digest(self, plainText):
        plainText += self.header
        plainText = self.padding.pad(plainText)
        return self.encryptor.update(plainText)

In [172]:
blockSize = 16
oracle = Oracle(blockSise=blockSize)
oracle.digest(b'hola que tal')

b'\x8e\xf3\xd3}\xde\x91V\'5\x8a\xd6}\xff:\xd0C\x9a\xf4\xbd\xe8\x81\xa0\x92\x92X\x13\x91y\xa7*\xfe\xb7-\x9c\xf0\x9c\xd7;\xc4\xb2\xe6\xe0\xba\xb8/au\xbb>\x04\xb9\'\x14qC\\\xea\xb8\xefIj9\x14\x9c>\xdc\x11\xe1L+\xe8\x0fHS\xe2(\x95\xe9[7\x95!D\xd1>\x81\x91\xb3\xfe\x89\xb6I_\xe3\t5/y\xc0\x93a\x81\x05[\x8e\x01\x85\x1a2#\x1a)\xcb\xb3_\x9c*\xce\x8dl\xd9\xba\xb9\xee\x0b|\xbe:\x17\xb0C\x02\x8e\x88>SI\xbe\x19\\a\xac\xee]2p`\x82\xfd\x85"\xc8p3{\x1du\x8a\xaf1'

In [173]:
stop = False
startingLen = len(oracle.digest(b''))
i = 1
while not stop:
    newLen = len(oracle.digest(b'A'*i))
    if newLen != startingLen:
        stop = True
    else:
        i += 1
blockSizeFound = newLen - startingLen
print(f'Padding length = {blockSizeFound}')

Padding length = 16


In [174]:
from utils import check_repetitions_ECB
print(f"ECB mode? {check_repetitions_ECB(oracle.digest(b'A'*blockSizeFound*2), blockSizeFound)}")

ECB mode? True


In [239]:
def crack_ecb_oracle(oracle, crackedBytes, blockSizeFound):
    plainText = b'A'* (blockSizeFound - 1 - len(crackedBytes) % blockSizeFound)
    cyphertext = oracle.digest(plainText)

    plainText = b'A'* (blockSizeFound - 1 - len(crackedBytes) % blockSizeFound) + crackedBytes
    comparisonLen = int(blockSizeFound * (int(len(crackedBytes)/blockSizeFound) + 1))
    for i in range(256):
        byte = int.to_bytes(i, length=1, byteorder='big')
        if oracle.digest(plainText + byte)[:comparisonLen] == cyphertext[:comparisonLen]:
            return byte
    return b''

In [240]:
decodedText = b''
for _ in range(len(oracle.digest(b''))):
    decodedText += crack_ecb_oracle(oracle, decodedText, blockSizeFound)

In [243]:
print(decodedText.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



In [246]:
print(oracle.header.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



# 13

In [1]:
text = b'foo=bar&baz=qux&zap=zazzle'

def parsing_routine(text):
    data = text.split(b'&')
    return {data.split(b'=')[0]: data.split(b'=')[1] for data in text.split(b'&')}

parsing_routine(text)

{b'foo': b'bar', b'baz': b'qux', b'zap': b'zazzle'}

In [2]:
email = b"foo@bar.com"

def profile_for(email):
    assert (b'&' not in email) | (b'=' not in email),\
        "'&' and '=' characters not allowed"
    return b'email='+email+b'&uid=10&role=user'

parsing_routine(profile_for(email))

{b'email': b'foo@bar.com', b'uid': b'10', b'role': b'user'}

In [3]:
not_allowed = b"foo@bar.com&role=admin"
parsing_routine(profile_for(not_allowed))

AssertionError: '&' and '=' characters not allowed

In [4]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from utils import PKCS7

class parsingRoutine:
    def __init__(self, key):
        self.key = key
        self.cipher = Cipher(
            algorithm=algorithms.AES(self.key),
            mode=modes.ECB()
            )
        self.decryptor = self.cipher.decryptor()
        self.padding = PKCS7(block_size=len(self.key))
        
    def digest(self, cipherText):
        paddedPlainText = self.decryptor.update(cipherText)
        plainText = self.padding.unpad(paddedPlainText)
        return parsing_routine(plainText)
    
    
class profileFor:
    def __init__(self, key):
        self.key = key
        self.cipher = Cipher(
            algorithm=algorithms.AES(self.key),
            mode=modes.ECB()
            )
        self.encryptor = self.cipher.encryptor()
        self.padding = PKCS7(block_size=len(self.key))
    
    def make_profile(self, user):
        profileUser = profile_for(user)
        padProfileUser = self.padding.pad(profileUser)
        return self.encryptor.update(padProfileUser)

In [5]:
key = os.urandom(16)
profile4 = profileFor(key)
parsing = parsingRoutine(key)

In [6]:
cipherProfile = profile4.make_profile(email)
print(cipherProfile)

parsing.digest(cipherProfile)

b'\xebom][\xb8\xae\xf96\xef\xb3\xc2\xb9R\x07\xb7\xcb\xd58\x0fW\xce\x9b\x18\xf2\x1c\xdd\xfc\x85(1\xe4\x97x\x9c\xe4\x91(z\xd7L\xb2_\x94}\xbd\xd8S'


{b'email': b'foo@bar.com', b'uid': b'10', b'role': b'user'}

In [7]:
from utils import find_padding_length
blockSize = find_padding_length(profile4.make_profile)
print(blockSize)

16


In [8]:
from utils import check_repetitions_ECB
cipherText = profile4.make_profile(b'A'*blockSize*3)
check_repetitions_ECB(cipherText, blockSize)

True

In [9]:
email = b'@gmail.com'
userCharsUntilNewBlock = 1
stop = False
defLen = len(profile4.make_profile(email))
while not stop:
    newLen = len(profile4.make_profile(
        b'A'*userCharsUntilNewBlock+email
    ))
    if defLen != newLen:
        stop = True
    else:
        userCharsUntilNewBlock += 1

In [10]:
# We need 'role=user' falling on the last block so the user mail has to be 
# the following bytes long:
userLength = (userCharsUntilNewBlock + len(b'role=user')) % blockSize
email = b'A' * userLength + email
print(email)

b'AAAAAAAA@gmail.com'


In [11]:
cipherProfile = profile4.make_profile(email)
print(cipherProfile)

b'\xea\x14Zx\xa8c\xd4Mi\xda\xac`f\x9b\x1d\xc5\x0f\\L\x8e\x00v\xc4\xeb\xaal3Qq\xc5\xdc\x8b\x8e\x99\x1f\x08\xde\x81Z\x92SGI\x0eo\x99\xf2\xda'


In [12]:
# Generating the cipher text that contains 'role=admin'
padding = PKCS7(blockSize)
adminText = padding.pad(b'role=admin')
print(adminText)

b'role=admin\x06\x06\x06\x06\x06\x06'


In [18]:
userLength4adminBlock = blockSize - len(b'email=')
adminCipherText = profile4.make_profile(
    b'a'*userLength4adminBlock + adminText)
adminCipherText = adminCipherText[blockSize:blockSize*2]
print(adminCipherText)


b'\xd0\xc1\x13\xa1\xa8\xd4\xa3nr\x1e\xb3\xc1g;\xce\x9d'


In [14]:
crackedCipherProfile = cipherProfile[:-blockSize] + adminCipherText
print(crackedCipherProfile)

b'\xea\x14Zx\xa8c\xd4Mi\xda\xac`f\x9b\x1d\xc5\x0f\\L\x8e\x00v\xc4\xeb\xaal3Qq\xc5\xdc\x8b\xd0\xc1\x13\xa1\xa8\xd4\xa3nr\x1e\xb3\xc1g;\xce\x9d'


In [15]:
parsing.digest(crackedCipherProfile)

{b'email': b'AAAAAAAA@gmail.com', b'uid': b'10', b'role': b'admin'}

# 14

In [20]:
import os
import base64
import numpy
from utils import PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

class Oracle:
    def __init__(self, blockSize):
        self.blockSize = blockSize
        self.key = os.urandom(self.blockSize)
        self.header = base64.b64decode("""
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
""")
        self.cipher = Cipher(
            algorithm=algorithms.AES(self.key),
            mode=modes.ECB()
            )
        self.encryptor = self.cipher.encryptor()
        self.padding = PKCS7(self.blockSize)
        self.prepend = os.urandom(
            int.from_bytes(os.urandom(1), byteorder='big')
            )


    def digest(self, plainText):
        plainText = self.prepend + plainText + self.header
        plainText = self.padding.pad(plainText)
        return self.encryptor.update(plainText)

In [21]:
oracle = Oracle(blockSize=16)

In [100]:
stop = False
emptyDigest = oracle.digest(b'')
i = 1
while not stop:
    newDigest = oracle.digest(b'A'*i)
    if len(emptyDigest) != len(newDigest):
        stop = True
    else:
        i += 1
blockSizeFound = len(newDigest) - len(emptyDigest)

print(f'Padding length = {blockSizeFound}')

Padding length = 16


In [109]:
from utils import check_repetitions_ECB
print(f"ECB mode? {check_repetitions_ECB(oracle.digest(b'A'*blockSizeFound*3), blockSizeFound)}")

ECB mode? True


In [107]:
oldDigest = oracle.digest(b'')
oldBlockDigest = [oldDigest[i:i+blockSizeFound] 
                  for i in range(0, len(oldDigest), blockSizeFound)]
newDigest = oracle.digest(b'A')
newBlockDigest = [newDigest[i:i+blockSizeFound] 
                  for i in range(0, len(oldDigest), blockSizeFound)]
reference = sum([i == j for i,j in zip(newBlockDigest, oldBlockDigest)])
for i in range(1, blockSizeFound+1):
    oldBlockDigest = newBlockDigest
    newDigest = oracle.digest(b'A' * (i+1))
    newBlockDigest = [newDigest[i:i+blockSizeFound]
                      for i in range(0, len(oldDigest), blockSizeFound)]
    countEqual = sum([i == j for i,j in zip(newBlockDigest, oldBlockDigest)])
    if reference != countEqual:
        break
nInputBytes = i
compStart = countEqual * blockSizeFound
print(f'Input bytes to fill the first blocks = {nInputBytes}')

Input bytes to fill the first blocks = 14


In [110]:
def crack_ecb_oracle(oracle, crackedBytes, blockSizeFound, fixedSizeNewBlock, compStart):
    plainText = b'A' * fixedSizeNewBlock
    plainText += b'A'* (blockSizeFound - 1 - len(crackedBytes) % blockSizeFound)
    cypherText = oracle.digest(plainText)

    plainText = b'A' * fixedSizeNewBlock
    plainText += b'A'* (blockSizeFound - 1 - len(crackedBytes) % blockSizeFound) + crackedBytes
    comparisonLen = compStart + int(blockSizeFound * int(len(crackedBytes)/blockSizeFound))
    oCipherBlock = cypherText[comparisonLen:comparisonLen+blockSizeFound]
    for i in range(256):
        byte = int.to_bytes(i, length=1, byteorder='big')
        iCipherText = oracle.digest(plainText + byte)
        iCipherBlock = iCipherText[comparisonLen:comparisonLen+blockSizeFound]
        if  iCipherBlock == oCipherBlock:
            return byte
    return b''

In [111]:
crack_ecb_oracle(
    oracle=oracle,
    crackedBytes=b'', 
    blockSizeFound=blockSizeFound, 
    fixedSizeNewBlock=nInputBytes, 
    compStart=compStart)

b'R'

In [113]:
decodedText = b''
for _ in range(len(oracle.digest(b''))):
    decodedText += crack_ecb_oracle(
        oracle=oracle,
        crackedBytes=decodedText, 
        blockSizeFound=blockSizeFound, 
        fixedSizeNewBlock=nInputBytes, 
        compStart=compStart
        )

In [116]:
print(decodedText.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



# 15

In [2]:
correctPadding = b"ICE ICE BABY\x04\x04\x04\x04"
incorrectPadding1 = b"ICE ICE BABY\x05\x05\x05\x05"
incorrectPadding2 = b"ICE ICE BABY\x01\x02\x03\x04"

In [19]:
class PKCS7:
    def __init__(self, block_size):
        self.block_size = block_size

    def pad(self, text):
        n = self.block_size - len(text) % self.block_size
        return text + bytes([n] * n)
    
    def validPad(self, text):
        text = text[-self.block_size:]
        n = text[-1]
        if text[-n:] == bytes([n]) * n:
            return True
        else:
            raise Exception('Invalid Padding')
    
    def unpad(self, text):
        if self.validPad(text):
            n = text[-1]
            return text[:-n]

In [20]:
blockSize = len(correctPadding)
padding = PKCS7(blockSize)
padding.unpad(correctPadding)

b'ICE ICE BABY'

In [21]:
padding.unpad(incorrectPadding1)

Exception: Invalid Padding

In [22]:
padding.unpad(incorrectPadding2)

Exception: Invalid Padding

# 16

In [1]:
from utils import PKCS7, AES_CBC
import os

BLOCK_SIZE = 16
KEY = b'secret_key:12345'
IV = os.urandom(BLOCK_SIZE)

def generate_chiper_text_oracle(plain_text: bytes) -> bytes:
    blockCipher = AES_CBC(
        key=KEY,
        IV=IV,
    )
    assert (b';' not in plain_text) | (b'=' not in plain_text),\
        "';' and '=' characters not allowed"
    prepend_str = b'comment1=cooking%20MCs;userdata='
    append_str = b';comment2=%20like%20a%20pound%20of%20bacon'
    plain_text = prepend_str + plain_text + append_str
    return blockCipher.encrypt(plain_text)

def admin_oracle(cipher_text: bytes) -> bool:
    blockCipher = AES_CBC(
        key=KEY,
        IV=IV,
    )
    plain_text = blockCipher.decrypt(cipher_text)
    return b';admin=true;' in plain_text

In [54]:
cipher_text = generate_chiper_text_oracle(b'')
print(cipher_text)
print(admin_oracle(cipher_text))

b'\xc3u\xda":\xabkF^\x11\xbcJ\x00\xcd\xa8\x0fs\x9a\xc9\xa6\xe7h\xafEO$]\xa8\xe3\xd1\x12Y@\xff;|\xbb\x1fA\x14\x92\x07Aa9?\'\xf5\x894\xeb\xcd/\xf93\x0e\xa9\x17\x86\xb9`\x0b\xf1=_\xda\xe2\x8c\xd7\xf7I\x13\x1cv\xc7\x15\xbd\x88\x8ds'
False


In [55]:
from utils import find_block_size
block_size = find_block_size(generate_chiper_text_oracle)
n_blocks = int(len(cipher_text)/block_size)
print('Number of blocks:', n_blocks)

Block Size = 16
Number of blocks: 5


In [107]:
cipher_text = generate_chiper_text_oracle(b'\x00' * block_size * n_blocks)

In [110]:
from utils import xor
inject = b';admin=true;'

pad = PKCS7(block_size)
inject = pad.pad(inject)
attack_block_n = n_blocks - 1
modify_block = \
    cipher_text[attack_block_n * block_size: (attack_block_n + 1) * block_size]
modified_block = xor(modify_block, inject)
cracked_cipher_text = (
    cipher_text[:attack_block_n * block_size]
    + modified_block
    + cipher_text[(attack_block_n + 1) * block_size:]
)

In [111]:
admin_oracle(cracked_cipher_text)

True